In [1]:
# 安装兼容库 tf_keras
!pip install tensorflow tf_keras pillow numpy pandas

import zipfile
import os

zip_file_name = "test_images.zip" 

if not os.path.exists("test_images"): 
    print("📦 正在解压图片...")
    try:
        with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
            zip_ref.extractall(".")
        print("✅ 解压完成！")
    except FileNotFoundError:
        print("❌ 没找到 zip 包！")
else:
    print("✅ 图片文件夹已存在。")

Defaulting to user installation because normal site-packages is not writeable
📦 正在解压图片...
❌ 没找到 zip 包！


In [2]:
import os
import numpy as np
import pandas as pd
import tf_keras as keras  # <--- 关键！我们要用这个兼容库！
from PIL import Image, ImageOps
from IPython.display import display

def test_my_model():
    # --- 配置区域 ---
    MODEL_PATH = r"keras_model.h5"
    LABELS_PATH = r"labels.txt"
    TEST_DIR = r"Test"
    # ----------------
    
    if not os.path.exists(MODEL_PATH) or not os.path.exists(LABELS_PATH):
        print("❌ 找不到文件！")
        return None

    print("🤖 正在加载模型... ")
    
    model = keras.models.load_model(MODEL_PATH, compile=False)
    
    class_names = [line.strip() for line in open(LABELS_PATH, "r").readlines()]
    results = []
    
    if not os.path.exists(TEST_DIR):
        print(f"❌ 找不到 {TEST_DIR}")
        return None

    for true_label_folder in os.listdir(TEST_DIR):
        folder_path = os.path.join(TEST_DIR, true_label_folder)
        if not os.path.isdir(folder_path) or true_label_folder.startswith('.'): continue
            
        print(f"👁️ 正在测试类别: {true_label_folder} ...")
        
        for img_file in os.listdir(folder_path):
            if not img_file.lower().endswith(('.png', '.jpg', '.jpeg')): continue
            
            img_path = os.path.join(folder_path, img_file)
            data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
            
            try:
                image = Image.open(img_path).convert("RGB")
                image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
                normalized_image_array = (np.asarray(image).astype(np.float32) / 127.5) - 1
                data[0] = normalized_image_array
            except Exception:
                continue
            
            # 预测
            prediction = model.predict(data, verbose=0)
            index = np.argmax(prediction)
            class_name = class_names[index]
            confidence = prediction[0][index]
            
            predicted_label = class_name[2:] if " " in class_name else class_name
            is_correct = "✅" if true_label_folder.lower() in predicted_label.lower() else "❌"
            
            results.append({
                "文件名": img_file,
                "真实类别": true_label_folder,
                "AI预测": predicted_label,
                "置信度": confidence, 
                "结果": is_correct
            })
            
    df = pd.DataFrame(results)
    if not df.empty:
        df['置信度展示'] = df['置信度'].apply(lambda x: f"{x:.2%}")
    
    return df

C:\Users\weirui\AppData\Roaming\Python\Python312\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [3]:
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
from IPython.display import display

# 运行刚才定义的测试函数
df_result = test_my_model()

if df_result is not None and not df_result.empty:
    print(f"\n🎉 测试完成！一共测试了 {len(df_result)} 张图片。")
    
    # --- 1. 基础准确率 ---
    correct_count = len(df_result[df_result['结果'] == '✅'])
    accuracy = correct_count / len(df_result)
    print(f"🏆 模型总准确率 (Accuracy): {accuracy:.2%}")
    
    # --- 2. 专业评测报告 ---
    print("\n📊 === 专业医疗级评测报告  ===")
    
    y_true = df_result['真实类别']
    y_pred = df_result['AI预测']
    
    # 把报告生成字典，方便改名
    report_dict = classification_report(y_true, y_pred, output_dict=True)
    
    # 把字典转成漂亮的表格
    report_df = pd.DataFrame(report_dict).transpose()
    
    # 给列名穿上“中文马甲”
    report_df.columns = [
        '精确率 (Precision)', 
        '召回率 (Recall)', 
        'F1分数 (F1-Score)', 
        '样本数 (Support)'
    ]
    
    new_index = []
    for idx in report_df.index:
        if 'cataract' in idx.lower():
            new_index.append('白内障 (Cataract)')
        elif 'normal' in idx.lower():
            new_index.append('正常 (Normal)')
        elif 'accuracy' in idx:
            new_index.append('总准确率 (Accuracy)')
        elif 'macro avg' in idx:
            new_index.append('宏平均 (Macro Avg)')
        elif 'weighted avg' in idx:
            new_index.append('加权平均 (Weighted Avg)')
        else:
            new_index.append(idx)
            
    report_df.index = new_index
    
    # 展示这个漂亮的表格！
    display(report_df)

    # --- 3. 补充特异度 (Specificity) ---
    # 混淆矩阵
    # 假设标签里有 'cataract' 和 'Normal' (注意大小写，代码会自动匹配)
    labels = list(set(y_true) | set(y_pred))
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    
    # 简单解释一下混淆矩阵
    print("\n🧐 混淆矩阵详情:")
    cm_df = pd.DataFrame(cm, index=[f"真实:{l}" for l in labels], columns=[f"预测:{l}" for l in labels])
    display(cm_df)

    print("---------------------------------------------------")
    print("👇 下面是【判断错误】的案例：")
    
    wrong_cases = df_result[df_result['结果'] == '❌'].sort_values(by='置信度', ascending=False)
    
    if len(wrong_cases) > 0:
        display(wrong_cases[['文件名', '真实类别', 'AI预测', '置信度展示', '结果']].head(20))
    else:
        print("太厉害了！居然没有一个错误的！")
        
    df_result.to_csv("AI_中英对照测试报告.csv", index=False, encoding="utf-8_sig")
    print("\n📁 完整报告已保存为 'AI_中英对照测试报告.csv'")
else:
    print("😐 好像没有测试到任何图片...")

🤖 正在加载模型... 


👁️ 正在测试类别: Cataract ...
👁️ 正在测试类别: Normal ...

🎉 测试完成！一共测试了 1573 张图片。
🏆 模型总准确率 (Accuracy): 98.54%

📊 === 专业医疗级评测报告  ===


,精确率 (Precision),召回率 (Recall),F1分数 (F1-Score),样本数 (Support)
白内障 (Cataract),0.985934,0.984674,0.985304,783.000000
正常 (Normal),0.984829,0.986076,0.985452,790.000000
总准确率 (Accuracy),0.985378,0.985378,0.985378,0.985378
宏平均 (Macro Avg),0.985381,0.985375,0.985378,1573.000000
加权平均 (Weighted Avg),0.985379,0.985378,0.985378,1573.000000



🧐 混淆矩阵详情:


,预测:Cataract,预测:Normal
真实:Cataract,771,12
真实:Normal,11,779


---------------------------------------------------
👇 下面是【判断错误】的案例：


,文件名,真实类别,AI预测,置信度展示,结果
606,372.png,Cataract,Normal,100.00%,❌
313,2356.jpg,Cataract,Normal,99.95%,❌
836,1242.jpg,Normal,Cataract,99.93%,❌
828,1211.jpg,Normal,Cataract,99.93%,❌
631,3874.jpg,Cataract,Normal,99.76%,❌
914,1580.jpg,Normal,Cataract,99.74%,❌
545,342.png,Cataract,Normal,99.39%,❌
528,3351.jpg,Cataract,Normal,99.38%,❌
875,1391.jpg,Normal,Cataract,99.33%,❌
481,311.png,Cataract,Normal,99.25%,❌



📁 完整报告已保存为 'AI_中英对照测试报告.csv'
